In [1]:
#PARAMETERS
testDataFile="/Users/eliotpbrenner/PycharmProjects/SparsityBoost/data/synthetic_examples/experiments/0/alarm1000.dat"
numTestVariable=37
numTestDataPts=1000

1000

In [2]:
# Imports
using Base.Test

In [3]:
type model
  nodes::Int64
  nval::Array{Int64,1}
  dataArray::Array{Int64,2}
end

In [4]:
type jointDistWithMarginals
  pdist::Array{Float64,2}
  p_A::Array{Float64,1}
  p_B::Array{Float64,1}
end

In [10]:
# Constructor which calculates the marginals
jointDistWithMarginals(pdist::Array{Float64,2}) = jointDistWithMarginals(pdist, vec(sum(pdist,1)), vec(sum(pdist,2)))

jointDistWithMarginals

In [11]:
jointDistWithMarginals([[0.25 0.25]; [ 0.25 0.25]])

jointDistWithMarginals(2x2 Array{Float64,2}:
 0.25  0.25
 0.25  0.25,[0.5,0.5],[0.5,0.5])

In [18]:
function readData(filePath::AbstractString)
  dataArray = readdlm(filePath, ' ', Int);
  nodes=size(dataArray)[2];
  nval=2*ones(Int, nodes)
  theModel = model(nodes,nval,dataArray);
end


readData (generic function with 3 methods)

In [19]:
# test readData function
m=readData(testDataFile)
@test size(m.dataArray) == (numTestDataPts, numTestVariable)

In [20]:
function verify(aJointDistributionWithMarginals::jointDistWithMarginals)
  sumA=sum(aJointDistributionWithMarginals.pdist,1)
  sumB=sum(aJointDistributionWithMarginals.pdist,2)
  @test_approx_eq(transpose(sumA),aJointDistributionWithMarginals.p_A)
  @test_approx_eq(sumB,aJointDistributionWithMarginals.p_B)
  @test_approx_eq(sum(sumA),1.0)
  return true
end

verify (generic function with 1 method)

In [22]:
goodDist=jointDistWithMarginals([[0.25 0.25]; [ 0.25 0.25]], [0.5; 0.5], [0.5; 0.5])
@test verify(goodDist) == true

In [24]:
#TODO: determine why this this test fails instead of succeeding!
badDist = jointDistWithMarginals([[0.25 0.25]; [ 0.25 0.25]], [0.6; 0.4], [0.5; 0.5])
@test_throws ErrorException verify(badDist)

ErrorException("assertion failed: |transpose(sumA) - aJointDistributionWithMarginals.p_A| <= 2.220446049250313e-12\n  transpose(sumA) = [0.5\n 0.5]\n  aJointDistributionWithMarginals.p_A = [0.6,0.4]\n  difference = 0.09999999999999998 > 2.220446049250313e-12")

In [25]:
function MI_term(i::Int, j::Int, dist::jointDistWithMarginals)
  return dist.pdist[i,j]*log(dist.pdist[i,j]/(dist.p_A[i]*dist.p_B[j])  )
end

MI_term (generic function with 1 method)

In [26]:
function MI(dist::jointDistWithMarginals)
  (a,b)=size(dist.pdist)
  return sum([(MI_term(i,j,dist)) for i=1:a, j=1:b])
end

MI (generic function with 1 method)

In [27]:
MI_term(1,1,goodDist)

0.0

In [28]:
@test_approx_eq(MI(goodDist),0)

In [31]:
t=0.01
anotherDist=jointDistWithMarginals([[0.25+t 0.25-t], [ 0.25-t 0.25+t]], [0.5; 0.5], [0.5; 0.5])
verify(anotherDist)
@test_approx_eq(MI(anotherDist), 0.0008002134699838133)

In [32]:
testNotInDict(elt::Int, SToVal::Dict) = try
    """TODO: figure out how to propagate y up the call stack if it's not a KeyError"""
    x=SToVal[elt] 
    warn("$elt is in $SToVal with value $x")
    throw(DomainError())
  catch y
    if isa(y, KeyError)
      return true
    else
      return y
    end
  end

testNotInDict (generic function with 1 method)

In [33]:
function conditionalDist(m::model,i::Int,j::Int,SToVal::Dict)
  """
  TODO: that S is a dict of integers and that both are valid for the model
  TODO: vectorize the computation of the pDist
  """
  @test i!=j
  @test testNotInDict(i, SToVal)
  @test testNotInDict(j, SToVal)
  selectedPoints = m.dataArray[reduce(&,[m.dataArray[:,condVar] .==SToVal[condVar] for condVar in keys(SToVal)]),:]
  numSelectedPoints = size(selectedPoints)[1]
  pDist = zeros(m.nval[i], m.nval[j])
  for k = 1:m.nval[i]
      for l = 1:m.nval[j]
          pDist[k,l] = size(selectedPoints[(selectedPoints[:,i] .==k-1)&(selectedPoints[:,j] .==l-1), :])[1]/numSelectedPoints
    end
  end
  jointDistWithMarginals(pDist)
end

conditionalDist (generic function with 1 method)

In [34]:
SToVal = Dict(3=>0, 4=>1)
cd = conditionalDist(m, 1, 2, SToVal)

jointDistWithMarginals(2x2 Array{Float64,2}:
 0.304239  0.316708
 0.15212   0.226933,[0.456359102244389,0.543640897755611],[0.6209476309226932,0.3790523690773067])